# Experimenting with Cleaning, Clustering & Summarization Pipelines

### To do (technical)
- Implement date windows on my corpus loader function

In [10]:
import os
import re
import json

import numpy as np
import pandas as pd

from lib.helper import *
import lib.embedding_models as reps

from importlib import reload

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "C:/Users/Martin/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "bing"

## 0.  Build Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

### First, get a list of all the news dumps created so far

In [3]:
corpus = corpus_loader(storage_path, corpus_type)

In [4]:
corpus.head()

,date,link,origin,retrieval_timestamp,source_url,summary,title
0,2019-09-05T17:32:00.0000000Z,https://www.desmoinesregister.com/story/opinio...,bing_news_api,2019-09-05 21:35:05.105002,www.desmoinesregister.com,<b>Immigration</b> raids are as devastating to...,<b>Immigration</b> raids are as devastating to...
1,2019-09-05T20:06:00.0000000Z,https://www.philstar.com/the-freeman/cebu-news...,bing_news_api,2019-09-05 21:35:05.105002,www.philstar.com,"CEBU, Philippines — The Department of Justice ...",<b>Immigration</b> told to monitor convicts: L...
2,2019-09-05T17:21:00.0000000Z,https://www.irishtimes.com/news/crime-and-law/...,bing_news_api,2019-09-05 21:35:05.106001,www.irishtimes.com,Three members of an organised crime gang were ...,Three people jailed for assisting illegal <b>i...
3,2019-09-05T20:12:00.0000000Z,https://www.correctionsone.com/immigration-det...,bing_news_api,2019-09-05 21:35:05.106001,www.correctionsone.com,"NATCHEZ, Miss. — A privately run prison in Mis...",<b>Immigration</b> agency sets new contract wi...
4,2019-09-05T19:27:00.0000000Z,https://www.nzherald.co.nz/nz/news/article.cfm...,bing_news_api,2019-09-05 21:35:05.106001,www.nzherald.co.nz,A licensed <b>immigration</b> adviser has been...,<b>Immigration</b> adviser censured and fined ...


In [5]:
corpus.shape

(4487, 7)

## 1. Clean Text

By default, I'm not going to clean out all punctuation because I think it informs the POS tagger to some degree.

In [6]:
corpus['clean_text'] = corpus[['title', 'summary']].apply(lambda x: clean_text('.  '.join(x)), axis=1)

In [7]:
corpus['clean_text'].head()

0    Immigration raids are as devastating to commun...
1    Immigration told to monitor convicts: Lookout ...
2    Three people jailed for assisting illegal immi...
3    Immigration agency sets new contract with Miss...
4    Immigration adviser censured and fined for far...
Name: clean_text, dtype: object

## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

In [43]:
reload(reps)

<module 'lib.embedding_models' from 'D:\\Martin\\Documents\\GitHub\\news_crow\\lib\\embedding_models.py'>

In [42]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
#with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
#    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
#        for line in infile:
#            outfile.write(line.encode('cp1252'))

In [ ]:
infersent = reps.InferSentModel(list(corpus['clean_text']),
                                list(corpus['clean_text']),
                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

Found 8692(/9130) words with w2v vectors
Vocab size : 8692


In [17]:
# Whereas this worked first time!
#glove = reps.GloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
embeddings = infersent.get_embeddings()

'hello'